In [ ]:
# Import libraries
import rasterio as rio
import os
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import earthpy as et

# Get data
et.data.get_data("spatial-vector-lidar")
os.chdir(os.path.join(et.io.HOME,
                      "earth-analytics",
                      "data"))

In [ ]:
# Get help -- just a reminder to check the expected parameter input types.
help(gpd.clip)

## Open a shapefile Using Geopandas
Remember that we are working with VECTOR data this week which is different than raster data in it's structure. 
We will use GeoPandas for vector data. Geopandas data structures are very similar to pandas! they have additional spatial attributes associated with them. 


In [ ]:
base_data_path = os.path.join("spatial-vector-lidar")
cali_data_path = os.path.join(base_data_path, "california")

plot_centroid_path = os.path.join(cali_data_path,
                                  "neon-sjer-site", "vector_data", "SJER_plot_centroids.shp")

# Open a shapefile using geopandas
sjer_plot_locations = gpd.read_file(plot_centroid_path)

# View data attributes
sjer_plot_locations.head()

Explore your data.
You will find many of the same spatial attributes in vector data as you did 
raster data. 

In [ ]:
type(sjer_plot_locations)

In [ ]:
# View the spatial extent of the data
sjer_plot_locations.total_bounds

In [ ]:
sjer_plot_locations.crs

In [ ]:
# Check to see if your data are points, line or polygons
sjer_plot_locations.geom_type

In [ ]:
# You can plot directly using the .plot() method
sjer_plot_locations.plot()

In [ ]:
# Plot using Matplotlib
# Note that this works well with one layer
# But for multiple layers if you want a legend, you
# will want to create a loop to customize colors and labels
fig, ax1 = plt.subplots()
sjer_plot_locations.plot(ax=ax1,
                         column="plot_type",
                         legend=True,
                         cmap="Set2")
plt.show()

In [ ]:
# View object shape
sjer_plot_locations.shape

In [ ]:
np.unique(sjer_plot_locations.plot_type)

To plot several layers with custom symbology, you can create a loop.

In [ ]:
# Create a symbology dictionary that maps "type" to color. In this case the types are trees, grass and soil
points_symb = {'trees': 'chartreuse',
               'grass': 'darkgreen',
               'soil': 'burlywood'}

# Plot Data
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_axis_off()

# Add the plot points in sets grouped by plot_type
for ctype, sjer_plot in sjer_plot_locations.groupby('plot_type'):
    color = points_symb[ctype]
    label = ctype
    sjer_plot.plot(color=color,
                   ax=ax,
                   label=label,
                   markersize=50)

# For your homework you'll want to add roads to this plot.
# Be sure the data are in the correct CRS prior to plotting to ensure things line up!

# Add a legend
ax.legend()
plt.show()

## Challenge - break down the Plot Loop Above

Break down the  for loop above to figure out what each object in the loop  is

In [ ]:
# Play around with this
for an_object in sjer_plot_locations.groupby('plot_type'):
    an_object

an_object

In [ ]:
for cat, df in sjer_plot_locations.groupby('plot_type'):
    cat

cat

## Reproject Vector Data in Python

Next let's have a look at the roads data. Remember that you are creating a map of roads and points together. 

In [ ]:
# Import the data
sjer_roads_path = os.path.join(cali_data_path,
                               "madera-county-roads",
                               "tl_2013_06039_roads.shp")
sjer_roads = gpd.read_file(sjer_roads_path)

sjer_aoi_path = os.path.join(cali_data_path,
                             "neon-sjer-site",
                             "vector_data",
                             "SJER_crop.shp")

sjer_aoi = gpd.read_file(sjer_aoi_path)

# Are the crs' the same for the two datasets?
sjer_aoi.crs, sjer_roads.crs

In [ ]:
# Formally that they are the same (or not)
try:
    assert sjer_aoi.crs == sjer_roads.crs
    print("The CRS' for both datasets are the same. You can clip the data")
except AssertionError as message:
    print("The CRS' are not the same. Looks like you need to reproject one of the datasets to clip.")

 Notice that if you attempt to plot two datasets that are in different CRS' you will run into issues.
 It technically "works" but the data do not line up properly

In [ ]:
# Plot with the data in diff CRS
fig, ax1 = plt.subplots()
sjer_aoi.plot(ax=ax1)
sjer_roads.plot(ax=ax1)
plt.show()

In [ ]:
# Reproject the data by typing out the CRS
sjer_aoi_wgs84 = sjer_aoi.to_crs(epsg=4269)

In [ ]:
# Easier way if you have a layer that you want to match the CRS of
sjer_aoi_4269 = sjer_aoi.to_crs(sjer_roads.crs)

sjer_aoi_4269.crs, sjer_roads.crs

In [ ]:
# Formally that they are the same (or not)
try:
    assert sjer_aoi_4269.crs == sjer_roads.crs
    print("The CRS' for both datasets are the same. You can clip the data")
except AssertionError as message:
    print("The CRS' are not the same. Looks like you need to reproject one of the datasets to clip.")

Now the data line up properly! Now you can clip the data!
A nice way to understand CRS's is to look at the numeric values associated with the x and y locations. you will
then better understand why a plot looks off (like the example above when you tried to plot two datasets in two different CRS'. They are data from the same location but the CRS differences cause issues with processing the data together.

In [ ]:
# plot with the data in diff CRS
fig, ax = plt.subplots(figsize=(10, 10))
# zorder to adjust the order
sjer_roads.plot(ax=ax)
sjer_aoi_4269.plot(ax=ax,
                   color="black",
                   zorder=10)
plt.show()

## Clip Data
Sometimes you will hear clip and sometimes you will here crop. the idea here is similar to what you discovered with the raster data. you can clip the data to the extent of another dataset. This is nice when you want to only work with a smaller amount of data. It can reduce processing time. It also can make a map look nicer if you are focused only on one particular region but have data outside of the region. 

In [ ]:
clipped_roads = gpd.clip(sjer_roads, sjer_aoi_wgs84)
clipped_roads.plot()

In [ ]:
# Import data
country_boundary_us_path = os.path.join(base_data_path,
                                        "usa",
                                        "usa-boundary-dissolved.shp")
country_boundary_us = gpd.read_file(country_boundary_us_path)

state_boundary_us_path = os.path.join(base_data_path,
                                      "usa",
                                      "usa-states-census-2014.shp")
state_boundary_us = gpd.read_file(state_boundary_us_path)

pop_places_path = os.path.join(base_data_path,
                               "global",
                               "ne_110m_populated_places_simple",
                               "ne_110m_populated_places_simple.shp")
pop_places = gpd.read_file(pop_places_path)

# Do the data have the same crs?
country_boundary_us.crs, state_boundary_us.crs, pop_places.crs

In [ ]:
f, ax = plt.subplots()
pop_places.plot(ax=ax)
state_boundary_us.plot(ax=ax,
                       color="black")
plt.show()

In [ ]:
pop_places_clip = gpd.clip(pop_places, country_boundary_us)

# Plot the data in 2 frames or axes
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
country_boundary_us.plot(ax=ax1,
                         color="y")
pop_places_clip.plot(ax=ax1)
pop_places.plot(ax=ax2)
country_boundary_us.plot(ax=ax2,
                         color="y")
ax1.set_title('Clipped data')
ax2.set_title('Unclipped data')
plt.show()

In [ ]:
state_boundary_us.plot()
# View dataframe
state_boundary_us.head()

## Dissolve States to Regions Using  Spatial  Attributes




In [ ]:
# Dissolve the data by the "region" column
us_regions = state_boundary_us.dissolve(by="region")

# Plot the dissolved data
us_regions.plot(color="black",
                figsize=(10, 10),
                linewidth=2,
                edgecolor="white")

plt.show()

In [ ]:
us_regions['coords'] = us_regions['geometry'].apply(
    lambda x: x.representative_point().coords[:])
us_regions['coords'] = [us_regions[0] for us_regions in us_regions['coords']]
us_regions

## OPTIONAL: Add  labels to your map

We won't review this in class but  you can see how the code below works if you wish.

In [ ]:
# Plot the dissolved data
f, ax = plt.subplots()
us_regions.plot(color="black",
                figsize=(10, 10),
                linewidth=2,
                edgecolor="white",
                ax=ax)
# This loop is ONLY if you want to  add labels
# You can ignore this code i f you dont wish to add labels to your map!
for idx, row in us_regions.reset_index().iterrows():
    plt.annotate(text=row.region,
                 xy=row.coords,
                 horizontalalignment='center',
                 color="white")
plt.show()

## Dissolve And Aggregate

When you dissolve the data, what happens to the attribute values in each column?
For each region you have 2 or more states that each have data. That data can
be summarized in different ways

You can control how the values are summarized using the `.agg` function.

In [ ]:
us_regions.head()

In [ ]:
# Aggregate the data  - calculate the mean  and sum for eachc olumn. Note that
# Some of th e columns are dropped in this  case because pythoond oesn't
# know how to summarize certain  column types
us_regions = state_boundary_us.dissolve(by="region",
                                        aggfunc=["mean", "sum"])
us_regions.head()

In [ ]:
# Create a dictionary that maps a column header to a summary type
# This allows you to be more specific about how each column is summarized
# If you wnated to summarize "ALAND" by mean and sum you can use a dictionary / list entry like this:
# "ALAND": ["mean", "sum"]

sum_type = {"ALAND": "mean",
            "AWATER": "sum",
            # Grab the first value and assign to the column
            "STATEFP": "first"}

us_regions = state_boundary_us.dissolve(by="region",
                                        aggfunc=sum_type)
us_regions.head()

## Spatial Joins - Join Attributes From One  Shapefile To Another Shapefile

Use sjoin to join attributes to roads. In the example below, you want to assign 
each road a region name so you can summarize the roads data by region.

In [ ]:
#
roads_region = gpd.sjoin(roads_cl,  # The layer that you wish to add attributes to
                         us_regions,  # The layer that you wish to take attributes from
                         op="intersects") # If the geometries overlap,  then assign the attributes
roads_region.head(3)

In [ ]:
# Plot the roads  by region
# Note th at  each road is now assigned a region value based upon it's spatial location
roads_region.plot(column="index_right")

##  Open the Roads Layer - Complex  Geometries

Open the global roads layer. Note that this layer takes some time to plot  as
it has a lot of features in it.

In [ ]:
roads_path = os.path.join("spatial-vector-lidar",
                          "global",
                          "ne_10m_roads",
                          "ne_10m_roads.shp")
roads = gpd.read_file(roads_path)
roads.plot()

In [ ]:
%time

# Clip the roads layer
country_boundary_us_sim = country_boundary_us.simplify(.2,
                                                       preserve_topology=True)
roads_cl = gpd.clip(roads,
                    country_boundary_us_sim)

In [ ]:
%time

# Clip the roads layer when it's not simplified - notice how much slower it is.
# NOTE - this takes a LONG time to run  so proceed with caution
# roads_cl = gpd.clip(roads,
#                     country_boundary_us)

In [ ]:
# Plot the clipped roads
roads_cl.plot()